<a href="https://colab.research.google.com/github/webremake/AI-Talks/blob/main/recipes/use_cases/video_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/meta-llama/llama-recipes/blob/main/recipes/use_cases/VideoSummary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This demo app shows:
* How to use LangChain's YoutubeLoader to retrieve the caption in a YouTube video
* How to ask Llama 3 to summarize the content (per the Llama's input size limit) of the video in a naive way using LangChain's stuff method
* How to bypass the limit of Llama 3's 8k context length limit by using a more sophisticated way using LangChain's `refine` and `map_reduce` methods - see [here](https://python.langchain.com/docs/use_cases/summarization) for more info

We start by installing the necessary packages:
- [youtube-transcript-api](https://pypi.org/project/youtube-transcript-api/) API to get transcript/subtitles of a YouTube video
- [langchain](https://python.langchain.com/docs/get_started/introduction) provides necessary RAG tools for this demo
- [tiktoken](https://github.com/openai/tiktoken) BytePair Encoding tokenizer
- [pytube](https://pytube.io/en/latest/) Utility for downloading YouTube videos

In [3]:
!pip install langchain youtube-transcript-api tiktoken pytube replicate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 3.1 MB/s eta 0:00:00


Let's first load a long (2:47:16) YouTube video (Lex Fridman with Yann Lecun: Meta AI, Open Source, Limits of LLMs, AGI & the Future of AI) transcript using the YoutubeLoader.

In [4]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.4
    Uninstalling langchain-0.3.4:
      Successfully uninstalled langchain-0.3.4


In [5]:
from langchain.document_loaders import YoutubeLoader

loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=5t1vTLU7s40", add_video_info=True
)

In [6]:
!pip install --upgrade pytube

In [7]:
# load the youtube video caption into Documents
docs = loader.load()

PytubeError: Exception while accessing title of https://youtube.com/watch?v=5t1vTLU7s40. Please file a bug report at https://github.com/pytube/pytube

In [19]:
from youtube_transcript_api import YouTubeTranscriptApi
from langchain.schema import Document

# Указываем ID видео
video_id = "5t1vTLU7s40"

try:
    # Загружаем субтитры через YouTubeTranscriptApi
    transcript = YouTubeTranscriptApi.get_transcript(video_id)

    # Преобразуем субтитры в единый текстовый блок, чтобы он больше походил на цельный документ
    combined_text = "\n".join([item['text'] for item in transcript])

    # Создаем объект Document для всего текста, чтобы формат соответствовал требованиям
    docs = [
        Document(
            page_content=combined_text,
            metadata={"source": f"https://youtu.be/{video_id}"}
        )
    ]

    # Проверяем количество символов и выводим часть контента
    print(len(docs[0].page_content), docs[0].page_content[:300], len(docs))
except Exception as e:
    print(f"Ошибка при загрузке субтитров: {e}")

142689 - I see the danger of this
concentration of power
through proprietary AI systems
as a much bigger danger
than everything else.
What works against this
is people who think that
for reasons of security,
we should keep AI systems
under lock and key
because it's too dangerous
to put it in the hands of e 1


In [20]:
# check how many characters in the doc and some content
len(docs[0].page_content), docs[0].page_content[:300], len(docs)

(142689,
 "- I see the danger of this\nconcentration of power\nthrough proprietary AI systems\nas a much bigger danger\nthan everything else.\nWhat works against this\nis people who think that\nfor reasons of security,\nwe should keep AI systems\nunder lock and key\nbecause it's too dangerous\nto put it in the hands of e",
 1)

In [ ]:
# Смотрим полный текст субтитров
full_text = docs[0].page_content
print(full_text)  # Вывод полного текста субтитров в консоль

You should see 142689 returned for the doc character length, which is about 30k words or 40k tokens, beyond the 8k context length limit of Llama 3. You'll see how to summarize a text longer than the limit.

**Note:** We will be using [Replicate](https://replicate.com/meta/meta-llama-3-8b-instruct) to run the examples here. You will need to first sign in with Replicate with your github account, then create a free API token [here](https://replicate.com/account/api-tokens) that you can use for a while. You can also use other Llama 3 cloud providers such as [Groq](https://console.groq.com/), [Together](https://api.together.xyz/playground/language/meta-llama/Llama-3-8b-hf), or [Anyscale](https://app.endpoints.anyscale.com/playground) - see Section 2 of the Getting to Know Llama [notebook](https://github.com/meta-llama/llama-recipes/blob/main/recipes/quickstart/Getting_to_know_Llama.ipynb) for more info.

If you'd like to run Llama 3 locally for the benefits of privacy, no cost or no rate limit (some Llama 3 hosting providers set limits for free plan of queries or tokens per second or minute), see [Running Llama Locally](https://github.com/meta-llama/llama-recipes/blob/main/recipes/quickstart/Running_Llama2_Anywhere/Running_Llama_on_Mac_Windows_Linux.ipynb).

In [22]:
# enter your Replicate API token, or you can use local Llama. See README for more info
from getpass import getpass
import os

TOGETHERAI_API_TOKEN = getpass()
os.environ["TOGETHERAI_API_TOKEN"] = TOGETHERAI_API_TOKEN


··········


Next you'll call the Llama 3 70b chat model from Replicate because it's more powerful than the Llama 3 8b chat model when summarizing long text. You may also try Llama 3 8b model by replacing the `model` name with "meta/meta-llama-3-8b-instruct".

In [53]:
!pip install together langchain -U

from langchain_together import ChatTogether

llm = ChatTogether(
    model="meta-llama/Llama-3.2-11B-Vision-Instruct-Turbo",
    together_api_key=TOGETHERAI_API_TOKEN,
    temperature=0.0,
    top_p=1,
    max_tokens=1000
)

Once everything is set up, you can prompt Llama 3 to summarize the first 4000 characters of the transcript.

**Note:** The context length of 8k tokens in Llama 3 is roughly 6000-7000 words or 32k characters, so you should be able to use a number larger than 4000.

In [55]:
text = docs[0].page_content[:4000]
prompt = f"Make a summury for text below: {text}."
response = llm.generate([prompt])

# Извлекаем текст ответа из объекта response
summary = response.generations[0][0].text
print(summary)

Here is a summary of the text:

The conversation is with Yann LeCun, a renowned AI scientist, who shares his views on the future of artificial intelligence. He believes that the concentration of power through proprietary AI systems is a bigger danger than the technology itself. LeCun thinks that people are fundamentally good and that open-source AI can empower them. He is a proponent of open-sourcing AI development and has been critical of those who warn about the dangers of AGI (Artificial General Intelligence).

LeCun also discusses the limitations of autoregressive LLMs (Large Language Models), such as GPT-4 and LLaMA, which he believes are not a path towards human-level intelligence. He argues that these models lack essential characteristics of intelligent behavior, such as understanding the physical world, persistent memory, and planning. He also notes that the training data for these models is not as vast as it seems, and that it would take a person 170,000 years to read through 

You can try a larger text to see how the summary differs.

In [ ]:
text = docs[0].page_content[:10000]
summary = llm.invoke(f"Give me a summary of the text below: {text}.")
print(summary)

If you try the whole content which has over 142k characters, about 40k tokens, which exceeds the 8k limit, you'll get an empty result (Replicate used to return an error "RuntimeError: Your input is too long.").

In [18]:
# this will generate an empty result because the input exceeds Llama 3's context length limit
text = docs[0].page_content
summary = llm.invoke(f"Give me a summary of the text below: {text}.")
print(summary)

ReplicateError: ReplicateError Details:
title: Monthly spend limit reached
status: 402
detail: You've hit your monthly spend limit. You can change or remove your limit at https://replicate.com/account/billing#limits.

To fix this, you can use LangChain's `load_summarize_chain` method (detail [here](https://python.langchain.com/docs/use_cases/summarization)).

First you'll create splits or sub-documents of the original content, then use the LangChain's `load_summarize_chain` with the `refine` or `map_reduce` type.

Because this may involve many calls to Llama 3, it'd be great to set up a quick free LangChain API key [here](https://smith.langchain.com/settings), run the following cell to set up necessary environment variables, and check the logs on [LangSmith](https://docs.smith.langchain.com) during and after the run.

In [ ]:
import os
os.environ["LANGCHAIN_API_KEY"] = "your_langchain_api_key"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Video Summary with Llama 3"

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# we need to split the long input text
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)

In [ ]:
# check the splitted docs lengths
len(split_docs), len(docs), len(split_docs[0].page_content), len(docs[0].page_content)

The `refine` type implements the following steps under the hood:

1. Call Llama 3 on the first sub-document to generate a concise summary;
2. Loop over each subsequent sub-document, pass the previous summary with the current sub-document to generate a refined new summary;
3. Return the final summary generated on the final sub-document as the final answer - the summary of the whole content.

An example prompt template for each call in step 2, which gets used under the hood by LangChain, is:
```
Your job is to produce a final summary.
We have provided an existing summary up to a certain point:
<previous_summary>
Refine the existing summary (only if needed) with some more content below:
<new_content>
```

**Note:** The following call will make 33 calls to Llama 3 and genereate the final summary in about 10 minutes. The complete log of the the calls with inputs and outputs is [here](https://smith.langchain.com/public/7f23d823-926f-4874-bbd7-b509328a94bf/r).

In [ ]:
from langchain.chains.summarize import load_summarize_chain

chain = load_summarize_chain(llm, chain_type="refine")
chain.run(split_docs)

You can also set `chain_type` to [`map_reduce`](https://python.langchain.com/docs/modules/chains/document/map_reduce) to generate the summary of the entire content using the standard map and reduce method, which works behind the scene by first mapping each split document to a sub-summary via a call to LLM, then combines all those sub-summaries into a single final summary by yet another call to LLM.

**Note:** The following call takes about 3 minutes and all the calls to Llama 3 with inputs and outputs can be traced [here](https://smith.langchain.com/public/e54fad15-91ad-44a0-8d8f-f27a0d880b04/r).

In [ ]:
chain = load_summarize_chain(llm, chain_type="map_reduce")
chain.run(split_docs)

One final `chain_type` you can set is `stuff`, but it won't work with large documents because it stuffs all the split documents into one and uses it in a single prompt which exceeds the Llama 3 context length limit.

In [ ]:
# this will return nothing
chain = load_summarize_chain(llm, chain_type="stuff")
chain.run(split_docs)